In [ ]:
from pyspark import SparkContext,SparkConf
from random import randint
import random
import sys
import math
import itertools
import time

In [ ]:
'''
input_file_path = sys.argv[1]
output_file_path = sys.argv[2]
'''
input_file_path = "/Users/gopi/Desktop/Assignment3/yelp_train.csv"
output_file_path = "/Users/gopi/Desktop/Assignment3/output_1.csv"
#SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf().setMaster("local[*]").setAppName("assign-3-task-1").set("spark.executor.memory", "4g").set("spark.driver.memory", "4g")
sc = SparkContext(conf=conf)

In [ ]:

#a = random.sample(range(1, sys.maxsize - 1), num_hash_functions)
#b = random.sample(range(1, sys.maxsize - 1), num_hash_functions)
num_hash_functions = 30
a = [randint(1, 10001) for i in range(num_hash_functions)]
b = [randint(1, 10001) for i in range(num_hash_functions)]

def min_hashing(users_index, m):
    result = []
    for i in range(num_hash_functions):
        min_value = math.inf
        count = 0
        l = []
        for user_index in users_index:
            hash_value = ((a[i] * user_index + b[i]) % 99991) % m
            #if(hash_value == 0):
             #   print("a is: ", a[i], " b is : ", b[i], " user_index is :", user_index)
            if(count == 0):
                l.append(hash_value)
            if hash_value < min_value: min_value = hash_value
        #if count == 0: print(l)
        result.append(min_value)
    return result

def split_into_bands(hash_indices, bands):
    result = []
    size = int(len(hash_indices)/bands)
    for i in range(0,len(hash_indices), size):
        band = hash_indices[i : i+size]
        result.append((i,hash(tuple(band))))
    return tuple(result)

def verify_actual_similarity(candidate_pairs, business_user_map):
    result = []
    visited = set()
    for sorted_pair in candidate_pairs:
        #sorted_pair = tuple(sorted(pair))
        if sorted_pair not in visited:
            visited.add(sorted_pair)
            set1 = business_user_map[sorted_pair[0]]
            set2 = business_user_map[sorted_pair[1]]
            numerator = set1.intersection(set2)
            denominator = set1.union(set2)
            sim = len(numerator)/len(denominator)
            if(sim >= 0.5):
                result.append(sorted_pair)
    print(len(result))
    print(result[:5])
    return result
    


In [ ]:
start = time.time()
initial_rdd = sc.textFile(input_file_path)
first = initial_rdd.first()
initial_rdd = initial_rdd.filter(lambda line : line != first).map(lambda line : line.split(",")).map(lambda line: (line[0], line[1]))

user_index_rdd = initial_rdd.map(lambda x : x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1]))
user_index_map = user_index_rdd.collectAsMap()
user_count = len(user_index_map)
print(user_index_rdd.take(5))
business_index_rdd = initial_rdd.map(lambda x : x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1]))
business_index_map = business_index_rdd.collectAsMap()
print(business_index_rdd.take(5))

num_users = user_index_rdd.count()
#user_hash_index_rdd = user_index_rdd.map(lambda x : (x[1], get_hashed_index(x[1], num_users)))

business_user_rdd = initial_rdd.map(lambda x : (business_index_map[x[1]], user_index_map[x[0]])).groupByKey().mapValues(set)
business_user_map = business_user_rdd.collectAsMap()
#print(business_user_rdd.take(5))
signature_matrix_rdd = business_user_rdd.mapValues(lambda x : min_hashing(x, user_count))
#signature_matrix_rdd.saveAsTextFile("/Users/gopi/Desktop/Assignment3/temp_1.csv")
print(signature_matrix_rdd.count())
bands_rdd = signature_matrix_rdd.mapValues(lambda x : split_into_bands(x,num_hash_functions)).flatMap(lambda x : [(band_item, x[0]) for band_item in x[1]]).groupByKey().mapValues(set)
#print(bands_rdd.take(5))
candidate_items = bands_rdd.filter(lambda x : len(x[1]) > 1).map(lambda x : x[1]).flatMap(lambda x : [y for y in itertools.combinations(x,2)]).distinct().collect()
#candidate_items = set(candidate_items)
print("Length before verifying is : ", len(candidate_items))
#print(candidate_items[:5])
start_verify = time.time()
a = verify_actual_similarity(candidate_items, business_user_map)
print(len(a))
end = time.time()

print("time taken to verify is : ", (end - start_verify))
print("Total time is :", end - start)

In [ ]:
a = (213, 42)
b = 1
a = a + (b,)
print(a)
c = tuple()
c = c+a
print(c)